# Mesurer et afficher température pression hygrométrie

Utilisation des capteurs 
 * Press-Hygro-Temp : Adafruit BME280
 * LCD 2x16 JHD1313M1

Activité réalisée avec une raspberry Pi3 fonctionnant avec une image Debian Stretch/Buster fournie par l'IFÉ ENS de Lyon, disposant des librairies INTEL mraa et upm. en utilisant le bus i2c de la raspberry.

Sur le site de Adafruit https://learn.adafruit.com/adafruit-bme280-humidity-barometric-pressure-temperature-sensor-breakout/pinouts on lit que 4 broches doivent être connectées au raspberryPi.
* Vin - cest la broche d'alimentation : elle peut être reliée à 3,3V ou 5V (Nous la relierons au 3,3V dans cet exercice). 
* GND - broche à relier à une broche ground du RasberryPi
* SCK - c'est la broche d'horloge du bus I2C à relier à la broche 5 (SCL)
* SDI - c'est la broche de données du bus I2C à relier à la broche 3 (SDA)

<span style='color:red'>ATTENTION : des informations contradictoires sont disponibles sur internet sur la nécessité d'utiliser (ou pas) un [convertisseur de tension (levelshifter)](https://www.adafruit.com/product/757) (voltage level shifter). Par précaution  nous utilisons ce composant qui évite que les broches SCL et SDA soient directement reliées à la raspberry lorsque les capteurs sont alimentés sous une tension de 5V. La raspberry étant conçue pour recevoir au maximum en entrée une tension de 3,3V c'est cette tension qui est appliquée sur la sortie du convertisseur. Ainsi les voies SCL et SDA sont ajustées à cette valeur alors que l'entrée se fait en 5V fournis au(x) capteur(s). </span> 

![Capteur BME280 branché sur le bus I2C d'un RaspberryPi](images/RaspberryPi_BME280L.jpg)

![Capteur BME280 branché sur le bus I2C d'un RaspberryPi](images/RaspberryPi_BME280Detail.jpg)


## Mesurer puis écrire la température en 5 lignes 

Chaque ligne de texte ci-dessous est traduite dans le bloc suivant en une instruction dans un langage interprétable par la machine, à chaque alinéa correspond une instruction, la somme des instructions constitue un programme.

 1. Demander l'utilisation de la bibliothèque de programmes de base qui permet d'interagir et de piloter les capteurs sur le bus GPIO. Le nom de cette bibliothèque est mraa
 1. Une autre bibliothèque de programmes appelée upm fournit une collection de pilotes pour une grande variété de composants. Nous allons utiliser le pilote du capteur BME280 en python dont le nom est pyupm_bmp280 (py pour python, upm nom de la librairie, _ séparateur, bmp280 nom du capteur; le capteur bm**_e_** 280 est une **_évolution_** du BMP280)
 1. Déclarer le nom du capteur qu'on va utiliser (bme)
 1. Mettre à jour le capteur (on utilise la commande préprogrammée dans le pilote "update")
 1. Imprimer sur la sortie standard la température que l'on mesure (on utilise la fonction préprogrammée dans le pilote "getTemperature()" on appelle la température tempBME) 

In [1]:
import mraa
from upm import pyupm_bmp280 
bme = pyupm_bmp280.BME280(0,0x77)
bme.update()
print ("{0:.2f} °C".format(bme.getTemperature()))

21.67 °C


## Amélioration du programme en introduisant des variables
Pour améliorer la lisibilité et la généricité du programme on introduit des variables aui permettront de réutiliser le code dans d'autres contextes et avec d'autres capteurs

In [2]:
#Importation des bibliothèques du projet et simplification du nom des fonctions utilisées
import mraa
from upm import pyupm_bmp280 as bmp
# Utilisation de variables pour le numéro du bus i2c et l'adresse du capteur
bus = 0
addr = 0x77
#Déclaraton et nommage du capteur utilisé
bme = bmp.BME280(bus,addr)
#Initialisation du capteur avant la mesure
bme.update()
#Mesure de la température
tempBME = bme.getTemperature()
#Impression de la température sur la sortie standard
print ("{0:.2f} °C".format(tempBME))

20.92 °C


## Afficher la température mesurée sur  l'afficheur
Le bloc de code suivant permet d'afficher la température mesurée  par le capteur BME280 sur l'afficheur LCD 2x16.

In [3]:
#Importation des bibliothèques du projet et simplification du nom des fonctions utilisées
import mraa, time
from upm import pyupm_bmp280 as bmp
from upm import pyupm_jhd1313m1 as jdm
# Utilisation de variables pour le numéro du bus i2c, l'adresse des composants, la durée d'allumage...
bus = 0
bmeAddr = 0x77
lcdAddress = 0x3E
rgbAddress = 0x62
duration = 10
charDeg = 0
#Dessin du caractère degré
degree = (
    0b01100,
    0b10010,
    0b01100,
    0b00000,
    0b00000,
    0b00000,
    0b00000,
    0b00000)
#Déclaraton et nommage des composants utilisés
bme = bmp.BME280(bus,bmeAddr)
lcd = jdm.Jhd1313m1(bus,lcdAddress,rgbAddress)
#Initialisation du capteur avant la mesure
bme.update()
#Création du caractère degré
lcd.createChar(charDeg,degree)
#Positionnement du curseur sur l'écran
lcd.setCursor(1, 0)
#Mesure de la température
tempBME = bme.getTemperature()
#Affichage de la température sur la sortie standard
status = lcd.write("Temp : {0:.2f} ".format(tempBME))
status = lcd.write(chr(charDeg))
status = lcd.write("C ")
time.sleep(duration)
status = lcd.clear()
status = lcd.backlightOff()

## Afficher plusieurs mesures sur l'afficheur

Le bloc de code ci-dessous illustre les 3 mesures que  peut réaliser le capteur BME280,  les mesures sont affichées sur l'afficheur et le caractère spécial degré et ajouté.

In [13]:
#Importation des bibliothèques du projet et simplification du nom des fonctions utilisées
import mraa, time
from upm import pyupm_bmp280 as bmp
from upm import pyupm_jhd1313m1 as jdm
# Utilisation de variables pour le numéro du bus i2c, l'adresse des composants, la durée d'allumage...
bus = 0
bmeAddr = 0x77
lcdAddress = 0x3E
rgbAddress = 0x62
duration = 10
charDeg = 0
#Dessin du caractère degré
degree = (
    0b01100,
    0b10010,
    0b01100,
    0b00000,
    0b00000,
    0b00000,
    0b00000,
    0b00000)
#Déclaraton et nommage des composants utilisés
bme = bmp.BME280(bus,bmeAddr)
lcd = jdm.Jhd1313m1(bus,lcdAddress,rgbAddress)
#Initialisation du capteur avant la mesure
bme.update()
#Création du caractère degré
lcd.createChar(charDeg,degree)
#Positionnement du curseur sur l'écran ligne 1 case 0
lcd.setCursor(1, 0)
#Mesure de la température de la pression de l'hygrométrie
tempBME = bme.getTemperature()
pressBME = bme.getPressure() / 100.0
hygroBME = bme.getHumidity()
#Affichage de la température sur la sortie standard
lcd.write("Temp. : {0:.2f} ".format(tempBME))
lcd.write(chr(charDeg))
lcd.write("C Humid. : {0:.2f} %".format(hygroBME))
#Positionnement du curseur sur l'écran liigne 2 case 0
lcd.setCursor(2, 0)
status = lcd.write("Pression : {0:6.2f} hPa".format(pressBME))
for i in range(0,18,1) :
    lcd.scroll(True)
    time.sleep(0.8)
for i in range(0,18,1) :
    lcd.scroll(False)
    time.sleep(0.8)
time.sleep(5)
status = lcd.clear()
status = lcd.backlightOff()

## Ajouter de la couleur  et une fonction pour calculer la pression au niveau de la mer

La pression atmosphérique en un lieu dépend de son altitude. Pour pouvoir faire de la météorologie ou simplement comparer des mesures faites en différents endroits il faut partager une référence commune et le niveau de la mer est une solution simple et efficace. On convertit donc  notre mesure locale à la valeur qu'elle aurait  si nous étions au niveau de la mer. Il nous faut donc connaître notre altitude et importer des outils mathématiques. pour cela on ajoute la constante localAlt au programme et on importe la bibliothèque d'outils mathématiques de python.
Il est probable que nous ayons à réutiliser ce calcul  dans d'autres programmes et il serait malin de pouvoir réutiliser simplement les lignes de code que nous allons écrire. Plutôt que les noyer dans le programme nous allons _définir une fonction_ qui prendra un _paramètre_ (la pression mesurée m_Pressure) et qui nous _retournera_ la valeur à laquelle cette mesure correspond au niveau de la mer (sl_Pressure pour sea-level pressure). Le paramètre est une variable tout comme la valeur retournée.
La fonction est appelée ici convertSeaLevel() elle prend comme paramètre la variable mesurée par le capteur pressBME et fournit la variable décrivant  la pression au niveau de la mer seaLevelPress. On voit que les variables _locales_ utilisées par la fonction (m_Pressure, sl_Pressure) n'ont pas forcément le même nom que les variables _globales_ du programme principal (pressBME, seaLevelPress), cela permet de manipuler des noms de variables signifiants et cohérents avec leur contexte. Vous noterez que pouur souligner cette différence le nom des variables _locales à la fonction_ utilisent un système de séparation des mots (des "\_") différent de celui du programme principal ( le CamelCase).

In [39]:
#Importation des bibliothèques du projet et simplification du nom des fonctions utilisées
import mraa, math, time
from upm import pyupm_bmp280 as bmp
from upm import pyupm_jhd1313m1 as jdm
# Utilisation de variables pour le numéro du bus i2c, l'adresse des composants, la durée d'allumage...
bus = 0
bmeAddr = 0x77
lcdAddress = 0x3E
rgbAddress = 0x62
duration = 10
charDeg = 0
#Couleur du fond [r,g,b]
color = [164,32,0]
#Dessin du caractère degré
degree = (
    0b01100,
    0b10010,
    0b01100,
    0b00000,
    0b00000,
    0b00000,
    0b00000,
    0b00000)
#Altitude locale 
localAlt = 156.5
#Déclaraton et nommage des composants utilisés
bme = bmp.BME280(bus,bmeAddr)
lcd = jdm.Jhd1313m1(bus,lcdAddress,rgbAddress)
#Initialisation du capteur avant la mesure
bme.update()
#Création du caractère degré
lcd.createChar(charDeg,degree)
#Fonction de calcul de la pression au niveau de la mer
def convertSeaLevel (m_Pressure):
    sl_Pressure = m_Pressure / math.pow(1.0 - localAlt/44330, 5.255)
    return sl_Pressure
#Mesure de la température de la pression de l'hygrométrie
tempBME = bme.getTemperature()
pressBME = convertSeaLevel (bme.getPressure() / 100.0)
hygroBME = bme.getHumidity()
#Chargement de la couleur
lcd.setColor(color[0],color[1],color[2])
#Positionnement du curseur sur l'écran ligne du haut case 0
lcd.setCursor(2, 0)
#Affichage de la température sur l'écran
lcd.write("Temp.  : {0:7.2f}  ".format(tempBME))
lcd.write(chr(charDeg))
lcd.write("C  ")
#Positionnement du curseur sur l'écran liigne du bas case 0
lcd.setCursor(1, 0)
#Affichage de la pression sur l'écran
status = lcd.write("Press. : {0:7.2f} hPa".format(pressBME))
time.sleep(1)
#Déplacement du texte droite/gauche
for i in range(0,5,1) :
    lcd.scroll(True)
    time.sleep(0.8)
for i in range(0,5,1) :
    lcd.scroll(False)
    time.sleep(0.8)
#Positionnement du curseur sur l'écran ligne du haut case 0
lcd.setCursor(2, 0)
#Affichage de la température sur l'écran
lcd.write("Press. : {0:7.2f} hPa".format(pressBME))
#Positionnement du curseur sur l'écran liigne du bas case 0
lcd.setCursor(1, 0)
#Affichage de la pression sur l'écran
status = lcd.write("Humid. : {0:7.2f} %".format(hygroBME))
time.sleep(1)
#Déplacement du texte droite/gauche
for i in range(0,5,1) :
    lcd.scroll(True)
    time.sleep(0.8)
for i in range(0,5,1) :
    lcd.scroll(False)
    time.sleep(0.8)
time.sleep(5)
status = lcd.clear()
status = lcd.backlightOff()

## Faire une mesure périodique##

Lorsque l'on dispose d'un capteur et d'une alimentation suffisante il est bien sûr extrêmement intéressant de faire des mesures **en continu**. En fait on ne peut pas réellement mesurer de façon continue, on effectue des mesures les unes à la suite des autres en ménageant un temps (qui peut êetre très court) entre deux mesures pour transmettre le résultat et permettre au capteur d'être prêt pour la mesure suivante. La valeur du temps de pause est ici de 18 secondes valeur correspondant à la somme des fonctions sleep (dors pendant....) de la librairie des programmes de gestion du temps (qui est incluse au début du programme).

In [1]:
#Importation des bibliothèques du projet et simplification du nom des fonctions utilisées
import mraa, math, time
from upm import pyupm_bmp280 as bmp
from upm import pyupm_jhd1313m1 as jdm
# Utilisation de variables pour le numéro du bus i2c, l'adresse des composants, la durée d'allumage...
bus = 0
bmeAddr = 0x77
lcdAddress = 0x3E
rgbAddress = 0x62
duration = 10
charDeg = 0
#Couleur du fond [r,g,b]
color = [192,32,64]
#Dessin du caractère degré
degree = (
    0b01100,
    0b10010,
    0b01100,
    0b00000,
    0b00000,
    0b00000,
    0b00000,
    0b00000)
#Altitude locale 
localAlt = 156.5
#Déclaraton et nommage des composants utilisés
bme = bmp.BME280(bus,bmeAddr)
lcd = jdm.Jhd1313m1(bus,lcdAddress,rgbAddress)
#Initialisation du capteur avant la mesure
bme.update()
#Création du caractère degré
lcd.createChar(charDeg,degree)
#Fonction de calcul de la pression au niveau de la mer
def convertSeaLevel (m_Pressure):
    sl_Pressure = m_Pressure / math.pow(1.0 - localAlt/44330, 5.255)
    return sl_Pressure
#
# Boucle du programme
# while True donne une boucle infinie 
# for in range() fournit un nombre choisi d'itérations
# l'indice i peut-être utilisé pour contrôler les couleurs
#
#while True :
for i in range(0,4,1) :
    #Rotation des couleurs sur 3 valeurs pour signaler une nouvelle mesure
    color = [color[1],color[2],color[0]]
    #Mesure de la température de la pression de l'hygrométrie
    tempBME = bme.getTemperature()
    pressBME = convertSeaLevel (bme.getPressure() / 100.0)
    hygroBME = bme.getHumidity()
    #Chargement de la couleur
    lcd.setColor(color[0],color[1],color[2])
    #Positionnement du curseur sur l'écran ligne du haut case 0
    lcd.setCursor(2, 0)
    #Affichage de la température sur l'écran
    lcd.write("Temp.  : {0:7.2f}  ".format(tempBME))
    lcd.write(chr(charDeg))
    lcd.write("C  ")
    #Positionnement du curseur sur l'écran liigne du bas case 0
    lcd.setCursor(1, 0)
    #Affichage de la pression sur l'écran
    status = lcd.write("Press. : {0:7.2f} hPa".format(pressBME))
    time.sleep(1)
    #Déplacement du texte droite/gauche
    for i in range(0,5,1) :
        lcd.scroll(True)
        time.sleep(0.8)
    for i in range(0,5,1) :
        lcd.scroll(False)
        time.sleep(0.8)
    #Positionnement du curseur sur l'écran ligne du haut case 0
    lcd.setCursor(2, 0)
    #Affichage de la température sur l'écran
    lcd.write("Press. : {0:7.2f} hPa".format(pressBME))
    #Positionnement du curseur sur l'écran liigne du bas case 0
    lcd.setCursor(1, 0)
    #Affichage de la pression sur l'écran
    status = lcd.write("Humid. : {0:7.2f}   %".format(hygroBME))
    time.sleep(1)
    #Déplacement du texte droite/gauche
    for i in range(0,5,1) :
        lcd.scroll(True)
        time.sleep(0.8)
    for i in range(0,5,1) :
        lcd.scroll(False)
        time.sleep(0.8)
    status = lcd.clear()
status = lcd.clear()
status = lcd.backlightOff()